# Getting the Credit-score data from Kaggle

In [1]:
#import relevant libraries
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !kaggle datasets download -d parisrohan/credit-score-classification -p /content
# !unzip /content/credit-score-classification.zip -d /content

train_data = pd.read_csv('/content/train.csv')
#test_data = pd.read_csv('/content/test.csv')

# train_data = pd.read_csv('C:/Users/rajih/OneDrive/Documents/GitHub/E-commerce_Promotion_Analysis/Datasets/train.csv')
# test_data = pd.read_csv('C:/Users/rajih/OneDrive/Documents/GitHub/E-commerce_Promotion_Analysis/Datasets/test.csv')
#'/content/test.csv'
# Training and testing sets
print("Training data shape:", train_data.shape)
#print("Testing data shape:", test_data.shape)

Training data shape: (100000, 28)


<ipython-input-2-4cc0f87abf18>:9: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/content/train.csv')


# Exploratory Data Analysis

Study each attribute and its characteristics:

*   Name
*   Type (categorical, int/float, bounded/unbounded, text, structured, etc.)
*   % of missing values
*   Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
*   Possibly useful for the task?
*   Type of distribution (Gaussian, uniform, logarithmic, etc)

Quick look into the columns

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

Checking for duplicates

In [4]:
train_data.duplicated().sum()

0

Quick look into first few rows

In [5]:
train_data.sample(5)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
96184,0x24996,CUS_0xdcb,January,Lynchr,54,646-08-1772,Developer,11680.31,1125.359167,3,4,13,2,"Credit-Builder Loan, and Mortgage Loan",21,14,3.51,NaN,Standard,58.64,24.884787,22 Years and 4 Months,No,10.663655,47.82128322737825,Low_spent_Large_value_payments,324.05097886703226,Standard
41887,0x10b6d,CUS_0xc271,August,Emmotth,50,871-72-6391,Developer,35871.34,NaN,4,5,5,1,Student Loan,10,0,9.48,9.0,Good,525.01,37.878170,24 Years and 8 Months,No,25.049160,NaN,High_spent_Medium_value_payments,464.5590613145944,Standard
48104,0x12fde,CUS_0x9233,January,Jebw,17,467-66-3762,Media_Manager,18256.6,1662.383333,7,6,21,5,"Debt Consolidation Loan, Home Equity Loan, Deb...",46,17,15.65,7.0,Bad,4670.52,35.818982,1 Years and 7 Months,Yes,72.626594,156.11941280242928,Low_spent_Small_value_payments,227.4923265647731,Standard
59666,0x1739c,CUS_0x6ea4,March,Andrear,20,594-25-4221,Entrepreneur,28549.94,2217.161667,6,5,21,-100,"Credit-Builder Loan, Student Loan, Not Specifi...",60,15,11.64,10.0,_,1374.27,36.764332,8 Years and 1 Months,Yes,139.025914,254.08329666121696,Low_spent_Small_value_payments,118.60695609253996,Poor
96988,0x24e4a,CUS_0x599e,May,Victoriap,18,#F%$D@*&8,Musician,117785.2,9973.433333,0,1,3,2,"Personal Loan, and Not Specified",-4,0,1.75,3.0,_,621.66,35.318100,19 Years and 0 Months,No,166.362159,366.4182252877946,High_spent_Medium_value_payments,714.5629493218894,Good


Changing the types of columns

In [6]:
num_cols = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card","Interest_Rate","Num_of_Loan", "Delay_from_due_date","Num_of_Delayed_Payment","Changed_Credit_Limit", "Num_Credit_Inquiries","Outstanding_Debt","Credit_Utilization_Ratio","Total_EMI_per_month","Amount_invested_monthly",	"Monthly_Balance","Credit_History_Age"]
cat_cols = ["Occupation","Credit_Mix", "Spending_Level","Payment_Value","Payment_of_Min_Amount"]
target_col = ["Credit_Score"]

In [7]:
class DataCleaner:
    def __init__(self, dataframe, numeric_columns):
        self.dataframe = dataframe
        self.numeric_columns = numeric_columns

    def clean_numeric_columns(self):

        for col in self.numeric_columns:

            if self.dataframe[col].dtype == 'object':
                self.dataframe[col] = self.dataframe[col].str.extract(r'(\d+)')
                self.dataframe[col] = pd.to_numeric(self.dataframe[col], errors='coerce')
                self.dataframe[col] = self.dataframe[col].fillna(self.dataframe[col].median())

            else:
                self.dataframe[col] = self.dataframe[col].fillna(self.dataframe[col].median())

        return self.dataframe



cleaner = DataCleaner(train_data, num_cols)
df = cleaner.clean_numeric_columns()

% of missing values

In [8]:
df.isnull().sum()/float(len(df))*100

,0
ID,0.000
Customer_ID,0.000
Month,0.000
Name,9.985
Age,0.000
SSN,0.000
Occupation,0.000
Annual_Income,0.000
Monthly_Inhand_Salary,0.000
Num_Bank_Accounts,0.000


Looking into Null-values in Name

In [9]:
df['Name'].nunique()

10139

In [10]:
df[df['Name'].isnull()].head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,6.0,11.0,4.0,Good,809,23.933795,18.0,No,49.574949,24.0,High_spent_Medium_value_payments,358.0,Standard
17,0x161b,CUS_0x2dbc,February,NaN,34,486-85-3974,Engineer,143162,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",13,6.0,7.0,3.0,Good,1303,41.702573,17.0,No,246.992319,232.0,High_spent_Small_value_payments,998.0,Good
22,0x1620,CUS_0x2dbc,July,NaN,34,486-85-3974,Engineer,143162,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",8,6.0,7.0,3.0,Good,1303,38.068624,18.0,No,246.992319,263.0,High_spent_Small_value_payments,968.0,Standard
64,0x1662,CUS_0x4157,January,NaN,23,070-19-1622,Doctor,114838,9843.867500,2,5,7,100,"Personal Loan, Debt Consolidation Loan, and Au...",13,11.0,8.0,3.0,Good,1377,33.664554,21.0,No,226.892792,215.0,High_spent_Small_value_payments,802.0,Good
80,0x167a,CUS_0xa66b,January,NaN,40,221-30-8554,Teacher,33751,2948.605833,5,5,20,3,"Credit-Builder Loan, Personal Loan, and Auto Loan",16,20.0,11.0,4.0,Standard,1328,37.089076,19.0,NM,65.008174,117.0,High_spent_Medium_value_payments,362.0,Standard


 - Replacing Null values in Name with the name available for same customer id in another row

In [11]:
# Create a mapping of Customer ID to Name where Name is not null
id_name_mapping = df.loc[df['Name'].notnull(), ['Customer_ID', 'Name']].set_index('Customer_ID')['Name'].to_dict()

# Function to fill missing names using the mapping
def fill_missing_name(row):
    if pd.isnull(row['Name']):
        return id_name_mapping.get(row['Customer_ID'])
    return row['Name']

# Apply the function to fill missing names
df['Name'] = df.apply(fill_missing_name, axis=1)

#Check for any remaining missing names after applying the function
print(df['Name'].isnull().sum())


0


Looking into null-values for Type of Loan

In [12]:
df['Type_of_Loan'].nunique()

6260

In [13]:
df[df['Type_of_Loan'].isnull()].head(10)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
32,0x1632,CUS_0x1cdb,January,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,0,NaN,5,14.0,2.0,4.0,Standard,943,39.797764,30.0,Yes,0.0,276.0,!@9#%8,288.0,Standard
33,0x1633,CUS_0x1cdb,February,Deepaa,21,615-06-7821,Developer,35547,3093.745000,7,5,5,0,NaN,9,14.0,2.0,4.0,Standard,943,27.020360,30.0,NM,0.0,74.0,High_spent_Medium_value_payments,460.0,Standard
34,0x1634,CUS_0x1cdb,March,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,100,NaN,5,12.0,2.0,4.0,Standard,943,23.462303,30.0,Yes,0.0,173.0,Low_spent_Medium_value_payments,392.0,Standard
35,0x1635,CUS_0x1cdb,April,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,0,NaN,1,15.0,2.0,4.0,_,943,28.924954,30.0,Yes,0.0,96.0,High_spent_Medium_value_payments,438.0,Standard
36,0x1636,CUS_0x1cdb,May,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,0,NaN,9,17.0,2.0,4.0,_,943,41.776187,31.0,Yes,0.0,62.0,High_spent_Small_value_payments,482.0,Standard
37,0x1637,CUS_0x1cdb,June,Deepaa,21,615-06-7821,Developer,35547,3093.745000,7,5,5,0,NaN,5,15.0,2.0,4.0,Standard,943,29.217556,31.0,Yes,0.0,37.0,High_spent_Medium_value_payments,497.0,Standard
38,0x1638,CUS_0x1cdb,July,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,0,NaN,10,15.0,2.0,4.0,Standard,943,26.263823,31.0,Yes,0.0,181.0,Low_spent_Small_value_payments,394.0,Standard
39,0x1639,CUS_0x1cdb,August,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,100,NaN,1,15.0,2.0,4.0,Standard,943,25.862922,31.0,Yes,0.0,181.0,High_spent_Small_value_payments,364.0,Standard
40,0x163e,CUS_0x95ee,January,Np,31,612-70-8987,Lawyer,73928,3093.745000,4,1288,8,0,NaN,12,10.0,10.0,2.0,Good,548,39.962685,18.0,No,15015.0,98.0,High_spent_Large_value_payments,740.0,Good
41,0x163f,CUS_0x95ee,February,Np,31,612-70-8987,_______,73928,5988.705000,4,5,8,0,NaN,8,7.0,10.0,2.0,Good,548,42.769864,32.0,NM,0.0,172.0,Low_spent_Medium_value_payments,705.0,Good


In [14]:
df[df['Type_of_Loan'].isnull() & (df['Num_of_Loan'] != 0)].head(10)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
34,0x1634,CUS_0x1cdb,March,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,100,NaN,5,12.0,2.0,4.0,Standard,943,23.462303,30.0,Yes,0.0,173.0,Low_spent_Medium_value_payments,392.0,Standard
39,0x1639,CUS_0x1cdb,August,Deepaa,21,615-06-7821,Developer,35547,2853.309167,7,5,5,100,NaN,1,15.0,2.0,4.0,Standard,943,25.862922,31.0,Yes,0.0,181.0,High_spent_Small_value_payments,364.0,Standard
180,0x170e,CUS_0xac86,May,Nickb,20,028-16-4402,Entrepreneur,106733,8873.427500,4,4,1,100,NaN,5,6.0,0.0,0.0,Good,76,28.820554,33.0,No,0.0,345.0,High_spent_Medium_value_payments,791.0,Good
327,0x17e9,CUS_0xb5ff,August,Nateg,33,060-81-1328,Media_Manager,12909,853.824583,5,5,13,100,NaN,22,15.0,7.0,3.0,Standard,602,23.579851,17.0,No,0.0,60.0,Low_spent_Medium_value_payments,305.0,Standard
355,0x1815,CUS_0xaedb,April,Olivia Oranr,19,272-47-1135,Musician,85554,7185.502500,4,2,3,100,NaN,12,7.0,6.0,3.0,Good,1095,31.360245,19.0,No,0.0,303.0,Low_spent_Large_value_payments,684.0,Good
356,0x1816,CUS_0xaedb,May,Olivia Oranr,19,272-47-1135,Musician,85554,7185.502500,1488,2,3,100,NaN,14,14.0,6.0,3.0,_,1095,41.661802,19.0,No,0.0,70.0,High_spent_Large_value_payments,887.0,Good
427,0x1881,CUS_0x9ce6,April,Ekr,43,#F%$D@*&8,Developer,123732,10416.026667,4,3,11,100,NaN,15,8.0,11.0,3.0,Good,1154,38.990635,27.0,No,0.0,532.0,Low_spent_Medium_value_payments,789.0,Standard
449,0x18a3,CUS_0x3f5b,February,Mikeh,25,#F%$D@*&8,Doctor,80108,6866.692500,5,3,17,100,NaN,28,19.0,14.0,1.0,Standard,997,26.173145,17.0,Yes,0.0,52.0,High_spent_Large_value_payments,874.0,Standard
712,0x1a2e,CUS_0x3861,January,Fiona Ortizx,53,212-32-2085,Engineer,17992,3093.745000,3,5,6,100,NaN,5,9.0,8.0,2.0,Good,565,22.380399,18.0,No,0.0,88.0,!@9#%8,348.0,Poor
953,0x1b97,CUS_0x567b,February,Marilynj,54,542-44-1751,Manager,80396,6623.738333,5,7,9,100,NaN,8,11.0,10.0,0.0,Good,1357,37.808974,26.0,No,0.0,482.0,Low_spent_Large_value_payments,450.0,Good


We can see some of the rows with empty Type_of_Load values with really high value of Num_of Loan. Changing the values of Num of Loans to zero where the type of loan is empty since these seem to be errors

In [17]:
df[df['Type_of_Loan'].isnull()]['Num_of_Loan'].value_counts().head()

,count
Num_of_Loan,
0,10930
100,417
50,2
911,2
439,1


Testing to see if all people with no loans have empty type of load column

In [18]:
df[df['Num_of_Loan'] == 0]['Type_of_Loan'].value_counts().head()

,count
Type_of_Loan,


Testing for people who do have type of loan in the column but have unreasonably high values of number of loans

In [19]:
df2 = df[~df['Type_of_Loan'].isnull()]

df2[df2['Num_of_Loan'] > 20]['Num_of_Loan'].value_counts().head()

,count
Num_of_Loan,
100,3460
1150,4
1228,3
430,3
1480,3


Testing on a customer level to see if its an error or genuine values

In [20]:
df2[df2['Name'] == 'Lauren Tarao' ].head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
31008,0xcbb2,CUS_0xedd,January,Lauren Tarao,30,888-82-5609,Engineer,125468,10445.703333,4,4,10,4,"Payday Loan, Credit-Builder Loan, Personal Loa...",23,13.0,10.0,4.0,Standard,1136,29.866322,19.0,Yes,338.092409,227.0,High_spent_Large_value_payments,718.0,Standard
31009,0xcbb3,CUS_0xedd,February,Lauren Tarao,30,888-82-5609,Engineer,125468,10445.703333,4,4,10,4,"Payday Loan, Credit-Builder Loan, Personal Loa...",15,16.0,10.0,4.0,Standard,1136,30.955492,19.0,Yes,338.092409,209.0,High_spent_Small_value_payments,757.0,Standard
31010,0xcbb4,CUS_0xedd,March,Lauren Tarao,30,888-82-5609,Engineer,125468,10445.703333,4,4,10,4,"Payday Loan, Credit-Builder Loan, Personal Loa...",18,10.0,5.0,4.0,Standard,1136,30.499955,19.0,Yes,338.092409,814.0,Low_spent_Medium_value_payments,171.0,Standard
31011,0xcbb5,CUS_0xedd,April,Lauren Tarao,30,888-82-5609,Engineer,125468,10445.703333,4,4,10,4,"Payday Loan, Credit-Builder Loan, Personal Loa...",18,13.0,10.0,4.0,Standard,1136,26.109106,19.0,Yes,338.092409,383.0,High_spent_Medium_value_payments,572.0,Standard
31012,0xcbb6,CUS_0xedd,May,Lauren Tarao,30,888-82-5609,Engineer,125468,3093.745000,4,4,10,4,"Payday Loan, Credit-Builder Loan, Personal Loa...",16,13.0,10.0,4.0,_,1136,26.271873,19.0,Yes,338.092409,339.0,High_spent_Medium_value_payments,616.0,Standard


Seems like an error. Let's see how many times people have had high value of loans in our dataset. Looks like we can drop these rows.

In [21]:
df2[df2['Num_of_Loan'] > 30]['Name'].value_counts().head()

,count
Name,
Caroline Valetkevitchj,5
Kevin Yaob,5
Miedemac,5
Kazunori Takadax,4
Rick Rothackern,4


Let's extract types of loan and one-hot encode it into our dataset

In [ ]:
# # Flatten all loan types into a single list and get unique loan types
# unique_loan_types = pd.unique(df['Type_of_Loan'].dropna().str.split(', ').sum())

# # Display unique loan types
# print(unique_loan_types[:10])


In [15]:
import pandas as pd

# Define stop words to remove
stop_words = {'and'}

# Function to clean Type_of_Loan while preserving full phrases
def clean_loan_types(loan_text):
    if pd.isna(loan_text):
        return loan_text  # Keep NaN values unchanged

    # Remove "and" and extra spaces while keeping multi-word phrases
    cleaned_loans = " ".join(word for word in loan_text.split() if word.lower() not in stop_words)

    return cleaned_loans.strip()

# Apply the function to clean the Type_of_Loan column
df['Type_of_Loan'] = df['Type_of_Loan'].apply(clean_loan_types)

# Get unique cleaned loan types
# unique_loan_types = df['Type_of_Loan'].dropna().unique()

# Display unique loan types
# print(unique_loan_types)


In [ ]:
# Show unique values of 'Num_of_Loan' along with their count for NaN Type_of_Loan rows
na_rows = df[df['Type_of_Loan'].isna()]
num_of_loan_counts = na_rows['Num_of_Loan'].value_counts()
# print(num_of_loan_counts)

# Fixing columns

## Name

In [16]:
# Replacing Null values in Name with the name available for same customer id in another row

# Create a mapping of Customer ID to Name where Name is not null
id_name_mapping = df.loc[df['Name'].notnull(), ['Customer_ID', 'Name']].set_index('Customer_ID')['Name'].to_dict()

# Function to fill missing names using the mapping
def fill_missing_name(row):
    if pd.isnull(row['Name']):
        return id_name_mapping.get(row['Customer_ID'])
    return row['Name']

# Apply the function to fill missing names
df['Name'] = df.apply(fill_missing_name, axis=1)

#Check for any remaining missing names after applying the function
print(df['Name'].isnull().sum())


0


#### Creating a function to handle numeric anomalies

In [17]:
def handle_outliers_with_last_valid(df, col_name, cutoff, id_col="Customer_ID"):

    # 1. Identify rows where col_name > cutoff
    outlier_rows = df[df[col_name] > cutoff]
    print(f"Rows over {cutoff} for {col_name} before fix: {len(outlier_rows)}")

    # 2. Attempt to replace with last valid value
    for index, row in outlier_rows.iterrows():
        cust_id = row[id_col]

        # Find the last occurrence (earlier index) for this cust_id where col_name <= cutoff
        last_valid_row = df[
            (df[id_col] == cust_id) &
            (df[col_name] <= cutoff) &
            (df.index < index)
        ].sort_index(ascending=False).head(1)

        # If found, replace current value with that valid value
        if not last_valid_row.empty:
            df.at[index, col_name] = last_valid_row[col_name].values[0]

    # 3. Check how many remain over cutoff
    still_over_cutoff = df[df[col_name] > cutoff]
    print(f"Rows still over {cutoff} for {col_name} after last-valid replacement: {len(still_over_cutoff)}")

    # 4. Cap any remaining outliers at cutoff
    df.loc[df[col_name] > cutoff, col_name] = cutoff

    # Final verification
    print(df[col_name].describe())

    return df

## Age
 - Max age seen as 8698, we assume there's data entry challenges. given avg age in the US is around 80, we allow for a higher threshold and cut off at 85.

In [18]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Age",
    cutoff=85,
    id_col="Customer_ID"
)

Rows over 85 for Age before fix: 2781
Rows still over 85 for Age after last-valid replacement: 393
count    100000.000000
mean         33.515710
std          11.222993
min          14.000000
25%          24.000000
50%          33.000000
75%          42.000000
max          85.000000
Name: Age, dtype: float64


## Annual_Income

In [19]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Annual_Income",
    cutoff=1000000,        # covers more than 99%
    id_col="Customer_ID"
)

Rows over 1000000 for Annual_Income before fix: 961
Rows still over 1000000 for Annual_Income after last-valid replacement: 139
count     100000.00000
mean       52024.91634
std        53250.74894
min         7005.00000
25%        19368.25000
50%        37085.00000
75%        71826.25000
max      1000000.00000
Name: Annual_Income, dtype: float64


## Monthly_salary

In [20]:
#repace anomalies with previous value for same consumer
threshold = 13500 #based on percentile


over_threshold = df[df["Monthly_Inhand_Salary"] > threshold]
print(f"Rows over {threshold} before fix: {len(over_threshold)}")


for index, row in over_threshold.iterrows():
    cust_id = row["Customer_ID"]

    # Find the last occurrence (earlier index) for the same Customer_ID with salary <= threshold
    last_valid_row = df[
        (df["Customer_ID"] == cust_id) &
        (df["Monthly_Inhand_Salary"] <= threshold) &
        (df.index < index)
    ].sort_index(ascending=False).head(1)

    # If found, replace the current outlier with that valid value
    if not last_valid_row.empty:
        df.at[index, "Monthly_Inhand_Salary"] = last_valid_row["Monthly_Inhand_Salary"].values[0]

# Check how many remain above threshold
still_over = df[df["Monthly_Inhand_Salary"] > threshold]
print(f"Rows over {threshold} after last-valid replacement: {len(still_over)}")

Rows over 13500 before fix: 1065
Rows over 13500 after last-valid replacement: 671


## Num_Bank_accounts

In [21]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Num_Bank_Accounts",
    cutoff=30,
    id_col="Customer_ID"
)

Rows over 30 for Num_Bank_Accounts before fix: 1307
Rows still over 30 for Num_Bank_Accounts after last-valid replacement: 163
count    100000.000000
mean          5.410950
std           2.782155
min          -1.000000
25%           3.000000
50%           6.000000
75%           7.000000
max          30.000000
Name: Num_Bank_Accounts, dtype: float64


## Num_Credit_card

In [22]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Num_Credit_Card",
    cutoff=20,
    id_col="Customer_ID"
)

Rows over 20 for Num_Credit_Card before fix: 2263
Rows still over 20 for Num_Credit_Card after last-valid replacement: 288
count    100000.00000
mean          5.57628
std           2.20729
min           0.00000
25%           4.00000
50%           5.00000
75%           7.00000
max          20.00000
Name: Num_Credit_Card, dtype: float64


## Interest_rate

In [23]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Interest_Rate",
    cutoff=40,         # given very high interest rates, and the fact that credit cards rarely exceed 35-40%. we will cap at 40% (which will still cover over 95%)
    id_col="Customer_ID"
)

Rows over 40 for Interest_Rate before fix: 2034
Rows still over 40 for Interest_Rate after last-valid replacement: 247
count    100000.000000
mean         14.595490
std           8.821211
min           1.000000
25%           7.000000
50%          13.000000
75%          20.000000
max          40.000000
Name: Interest_Rate, dtype: float64


## Num_of_Loan

In [24]:
#Since for most of the cases with Na values of Type_of_loan, number of loans is zero,
#we assume that these big numbers are error and we set number of loan to 0 and

# Set 'Type_of_Loan' to 'No Loan' where it is NaN
df.loc[df['Type_of_Loan'].isna(), 'Type_of_Loan'] = 'No Loan'

# Set 'Num_of_Loan' to 0 where 'Type_of_Loan' is 'No Loan'
df.loc[df['Type_of_Loan'] == 'No Loan', 'Num_of_Loan'] = 0

## Type_of_Loan

In [25]:
# Define stop words to remove
stop_words = {'and'}

# Function to clean Type_of_Loan while preserving full phrases
def clean_loan_types(loan_text):
    if pd.isna(loan_text):
        return loan_text  # Keep NaN values unchanged

    # Remove "and" and extra spaces while keeping multi-word phrases
    cleaned_loans = " ".join(word for word in loan_text.split() if word.lower() not in stop_words)

    return cleaned_loans.strip()

# Apply the function to clean the Type_of_Loan column
df['Type_of_Loan'] = df['Type_of_Loan'].apply(clean_loan_types)

# Get unique cleaned loan types
unique_loan_types = df['Type_of_Loan'].dropna().unique()

# Show unique values of 'Num_of_Loan' along with their count for NaN Type_of_Loan rows
na_rows = df[df['Type_of_Loan'].isna()]
num_of_loan_counts = na_rows['Num_of_Loan'].value_counts()

# Flatten all loan types into a single list and get unique loan types
unique_loan_types = pd.unique(df['Type_of_Loan'].dropna().str.split(', ').sum())

#Since for most of the cases with Na values of Type_of_loan, number of loans is zero,
#we assume that these big numbers are error and we set number of loan to 0 and

# Set 'Type_of_Loan' to 'No Loan' where it is NaN
df.loc[df['Type_of_Loan'].isna(), 'Type_of_Loan'] = 'No Loan'

# Set 'Num_of_Loan' to 0 where 'Type_of_Loan' is 'No Loan'
df.loc[df['Type_of_Loan'] == 'No Loan', 'Num_of_Loan'] = 0


<ipython-input-25-31fd808006b9>:25: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_loan_types = pd.unique(df['Type_of_Loan'].dropna().str.split(', ').sum())


## Num_of_Delayed_Payment

In [26]:
df = handle_outliers_with_last_valid(
    df=df,
    col_name="Num_of_Delayed_Payment",
    cutoff=30,        # Cap all values above 30 to 30 given it cover 99%
    id_col="Customer_ID"
)

Rows over 30 for Num_of_Delayed_Payment before fix: 736
Rows still over 30 for Num_of_Delayed_Payment after last-valid replacement: 90
count    100000.00000
mean         13.39690
std           6.05782
min           0.00000
25%           9.00000
50%          14.00000
75%          18.00000
max          30.00000
Name: Num_of_Delayed_Payment, dtype: float64


## Number of Loans

In [34]:
# Step 1: Identify rows where Num_of_Loan > 20000
high_emi_rows = df[df["Num_of_Loan"] > 30]

# Step 2: Iterate over these rows and check if previous and next values are the same
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Get previous and next index
    prev_index = index - 1
    next_index = index + 1

    # Check if both indices exist and belong to the same Customer_ID
    if (
        prev_index in df.index and
        next_index in df.index and
        df.at[prev_index, "Customer_ID"] == customer_id and
        df.at[next_index, "Customer_ID"] == customer_id
    ):
        # Check if previous and next values are the same
        prev_value = df.at[prev_index, "Num_of_Loan"]
        next_value = df.at[next_index, "Num_of_Loan"]

        if prev_value == next_value:
            df.at[index, "Num_of_Loan"] = prev_value  # Replace high value

# Step 1: Identify remaining rows where Num_of_Loan > 20000
high_emi_rows = df[df["Num_of_Loan"] > 30]

# Step 2: Iterate over these rows and find the last lowest valid EMI
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the last occurrence where Num_of_Loan is lower for this Customer_ID
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Num_of_Loan"] <= 30) &  # Only lower values
        (df.index < index)  # Only consider past records
    ].sort_index(ascending=False).head(1)  # Get the last occurrence

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Num_of_Loan"] = last_valid_row["Num_of_Loan"].values[0]


# Step 1: Identify remaining rows where Num_of_Loan > 20000
high_emi_rows = df[df["Num_of_Loan"] > 30]

# Step 2: Iterate over these rows and find the next lowest valid EMI
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the next occurrence where Num_of_Loan is lower for this Customer_ID
    next_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Num_of_Loan"] <= 30) &  # Only lower values
        (df.index > index)  # Only consider future records
    ].sort_index(ascending=True).head(1)  # Get the next occurrence

    # Replace if a valid next row is found
    if not next_valid_row.empty:
        df.at[index, "Num_of_Loan"] = next_valid_row["Num_of_Loan"].values[0]

## Num_Credit_Inquiries

In [28]:
# Replacing Num_Credit_Inquiries with last logical number of inquiries available (between 0-50)

# Step 1: Identify rows where Num_Credit_Inquiries > 100
high_inquiry_rows = df[df["Num_Credit_Inquiries"] > 100]

# Step 2: Iterate over these rows and find the last valid (0-30 range) inquiry count
for index, row in high_inquiry_rows.iterrows():
    customer_id = row["Customer_ID"]  # Use the correct column name for customer ID

    # Find the last occurrence where Num_Credit_Inquiries is between 0 and 30 for this Customer_ID
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Num_Credit_Inquiries"].between(0, 30)) &
        (df.index < index)  # Only consider past records
    ].sort_index(ascending=False).head(1)  # Get the last occurrence

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Num_Credit_Inquiries"] = last_valid_row["Num_Credit_Inquiries"].values[0]


# Replacing Num_Credit_Inquiries with next logical number (between 0-50) of inquiries available in case the last number is not available (Jan/Feb)

# Step 1: Identify remaining rows where Num_Credit_Inquiries > 100
high_inquiry_rows = df[df["Num_Credit_Inquiries"] > 100]

# Step 2: Iterate over these rows and find the next valid (0-50 range) inquiry count
for index, row in high_inquiry_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the next occurrence where Num_Credit_Inquiries is between 0 and 50 for this Customer_ID
    next_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Num_Credit_Inquiries"].between(0, 50)) &
        (df.index > index)  # Only consider future records
    ].sort_index(ascending=True).head(1)  # Get the next occurrence

    # Replace if a valid next row is found
    if not next_valid_row.empty:
        df.at[index, "Num_Credit_Inquiries"] = next_valid_row["Num_Credit_Inquiries"].values[0]

## Total_EMI_per_month

In [36]:
# Step 1: Identify rows where Total_EMI_per_month > 20000
high_emi_rows = df[df["Total_EMI_per_month"] > 150]

# Step 2: Iterate over these rows and check if previous and next values are the same
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Get previous and next index
    prev_index = index - 1
    next_index = index + 1

    # Check if both indices exist and belong to the same Customer_ID
    if (
        prev_index in df.index and
        next_index in df.index and
        df.at[prev_index, "Customer_ID"] == customer_id and
        df.at[next_index, "Customer_ID"] == customer_id
    ):
        # Check if previous and next values are the same
        prev_value = df.at[prev_index, "Total_EMI_per_month"]
        next_value = df.at[next_index, "Total_EMI_per_month"]

        if prev_value == next_value:
            df.at[index, "Total_EMI_per_month"] = prev_value  # Replace high value

# Step 1: Identify remaining rows where Total_EMI_per_month > 20000
high_emi_rows = df[df["Total_EMI_per_month"] > 150]

# Step 2: Iterate over these rows and find the last lowest valid EMI
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the last occurrence where Total_EMI_per_month is lower for this Customer_ID
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Total_EMI_per_month"] <= 150) &  # Only lower values
        (df.index < index)  # Only consider past records
    ].sort_index(ascending=False).head(1)  # Get the last occurrence

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Total_EMI_per_month"] = last_valid_row["Total_EMI_per_month"].values[0]


# Step 1: Identify remaining rows where Total_EMI_per_month > 20000
high_emi_rows = df[df["Total_EMI_per_month"] > 150]

# Step 2: Iterate over these rows and find the next lowest valid EMI
for index, row in high_emi_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the next occurrence where Total_EMI_per_month is lower for this Customer_ID
    next_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Total_EMI_per_month"] <= 150) &  # Only lower values
        (df.index > index)  # Only consider future records
    ].sort_index(ascending=True).head(1)  # Get the next occurrence

    # Replace if a valid next row is found
    if not next_valid_row.empty:
        df.at[index, "Total_EMI_per_month"] = next_valid_row["Total_EMI_per_month"].values[0]

## Monthly_Balance

In [30]:
# Step 1: Identify rows where Monthly_Balance is extremely high (> 50,000)
high_balance_rows = df[df["Monthly_Balance"] > 50000]  # Adjust condition if needed

# Step 2: Iterate over these rows and find the last valid (low) balance
for index, row in high_balance_rows.iterrows():
    customer_id = row["Customer_ID"]  # Ensure this column exists

    # Find the last occurrence where Monthly_Balance is < 50000 for this Customer_ID
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Monthly_Balance"] < 50000) &  # Only lower values
        (df.index < index)  # Only consider past records
    ].sort_index(ascending=False).head(1)  # Get the last occurrence

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Monthly_Balance"] = last_valid_row["Monthly_Balance"].values[0]


## Payment_of_Min_Amount

In [31]:
# Step 1: Identify rows where Payment_of_Min_Amount is "NM"
nm_rows = df[df["Payment_of_Min_Amount"] == "NM"]

# Step 2: Iterate over these rows and check if previous and next values are the same
for index, row in nm_rows.iterrows():
    customer_id = row["Customer_ID"]

    # Get previous and next index
    prev_index = index - 1
    next_index = index + 1

    # Check if both indices exist and belong to the same Customer_ID
    if (
        prev_index in df.index and
        next_index in df.index and
        df.at[prev_index, "Customer_ID"] == customer_id and
        df.at[next_index, "Customer_ID"] == customer_id
    ):
        # Check if previous and next values are the same (and not "NM")
        prev_value = df.at[prev_index, "Payment_of_Min_Amount"]
        next_value = df.at[next_index, "Payment_of_Min_Amount"]

        if prev_value == next_value and prev_value != "NM":
            df.at[index, "Payment_of_Min_Amount"] = prev_value  # Replace NM value

# Step 3: Identify remaining "NM" rows and replace with last valid (non-"NM") value
nm_rows = df[df["Payment_of_Min_Amount"] == "NM"]

for index, row in nm_rows.iterrows():
    customer_id = row["Customer_ID"]

    # Find the last occurrence where Payment_of_Min_Amount is not "NM"
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Payment_of_Min_Amount"] != "NM") &
        (df.index < index)
    ].sort_index(ascending=False).head(1)

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Payment_of_Min_Amount"] = last_valid_row["Payment_of_Min_Amount"].values[0]

# Step 4: Identify remaining "NM" rows and replace with next valid (non-"NM") value
nm_rows = df[df["Payment_of_Min_Amount"] == "NM"]

for index, row in nm_rows.iterrows():
    customer_id = row["Customer_ID"]

    # Find the next occurrence where Payment_of_Min_Amount is not "NM"
    next_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Payment_of_Min_Amount"] != "NM") &
        (df.index > index)
    ].sort_index(ascending=True).head(1)

    # Replace if a valid next row is found
    if not next_valid_row.empty:
        df.at[index, "Payment_of_Min_Amount"] = next_valid_row["Payment_of_Min_Amount"].values[0]


## Credit_Mix

In [32]:
# Identify rows where Payment_Behaviour == "!@9#%8"
strange_val_rows = df[df["Credit_Mix"] == "_"]

# Iterate over these rows
for index, row in strange_val_rows.iterrows():
    customer_id = row["Customer_ID"]

    # Find the last occurrence where Payment_Behaviour is not "!@9#%8"
    last_valid_row = df[
        (df["Customer_ID"] == customer_id) &
        (df["Credit_Mix"] != "_") &
        (df.index < index)
    ].sort_index(ascending=False).head(1)

    # Replace if a valid previous row is found
    if not last_valid_row.empty:
        df.at[index, "Credit_Mix"] = last_valid_row["Credit_Mix"].values[0]

In [45]:
# 1) Compute the minimum EMI <= 800 for each customer
min_emi_map = (
    df[df["Total_EMI_per_month"] <= 800]
    .groupby("Customer_ID")["Total_EMI_per_month"]
    .min()
    .to_dict()
)

# 2) For each row, if Total_EMI_per_month > 800, replace it with that customer's min EMI (if available)
for idx, row in df.iterrows():
    cust_id = row["Customer_ID"]
    if row["Total_EMI_per_month"] > 800:
        if cust_id in min_emi_map:  # only replace if we have a valid min
            df.at[idx, "Total_EMI_per_month"] = min_emi_map[cust_id]

In [46]:
df.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Auto Loan,Credit-Builder Loan,Personal Loan,Home Equity Loan,Not Specified,No Loan,Mortgage Loan,Student Loan,Debt Consolidation Loan,Payday Loan
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.00000,100000.000000
mean,33.515710,52024.91634,3984.951951,5.410950,5.57628,14.595490,3.53405,21.068780,13.39690,9.945490,5.807080,1425.722400,32.285173,17.969540,98.419696,614.454560,399.787930,0.305600,0.31728,0.311040,0.314000,0.316800,0.11408,0.313600,0.31040,0.31040,0.319440
std,11.222993,53250.74894,2890.521244,2.782155,2.20729,8.821211,2.45113,14.860104,6.05782,6.605945,4.010791,1155.131822,5.116875,7.935336,106.853303,1999.832038,210.842478,0.460663,0.46542,0.462921,0.464119,0.465231,0.31791,0.463958,0.46266,0.46266,0.466262
min,14.000000,7005.00000,303.645417,-1.000000,0.00000,1.000000,0.00000,-5.000000,0.00000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000
25%,24.000000,19368.25000,1792.084167,3.000000,4.00000,7.000000,2.00000,10.000000,9.00000,5.000000,3.000000,566.000000,28.052567,12.000000,28.482551,77.000000,271.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000
50%,33.000000,37085.00000,3093.745000,6.000000,5.00000,13.000000,3.00000,18.000000,14.00000,9.000000,6.000000,1166.000000,32.305784,18.000000,64.680312,135.000000,336.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000
75%,42.000000,71826.25000,5318.267500,7.000000,7.00000,20.000000,5.00000,28.000000,18.00000,14.000000,8.000000,1945.250000,36.496663,24.000000,139.122937,255.000000,463.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.00000,1.000000,1.00000,1.00000,1.000000
max,85.000000,1000000.00000,15204.633333,30.000000,20.00000,40.000000,29.00000,67.000000,30.00000,36.000000,93.000000,4998.000000,50.000000,33.000000,10356.000000,10000.000000,1602.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.00000,1.00000,1.000000


In [47]:
df.to_csv("cleaned_trained_data.csv", index=False)